# Análisis de la Base de Datos

En este script vamos a realizar un estudio de la base de datos que se nos proporciona. En él, veremos el número de apariciones de cada landmark en global y crearemos una clase encargada de la manipulación de los datos y de la creación del fichero *annotations.csv* que usaremos en el script para crear las particiones.

Primero importamos los paquetes necesarios:

In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

/home/alejandro/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


## Lectura de Datos

En primer lugar vamos a leer las imágenes del dataset junto con sus respectivos Landmarks y vamos a transformar las imágenes en blanco y negro a imágenes de tres canales. Crearemos un Data frame de Pandas con todos los datos.

In [2]:
import re
numbers = re.compile(r'(\d+)')

def show_landmarks(image, landmarks):
    """Show image with landmarks"""
    plt.imshow(image)
    plt.scatter(landmarks[:, 0], landmarks[:, 1], s=10, marker='.', c='r')
    plt.pause(0.001)  # pause a bit so that plots are updated

def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

def readLandmarks(landmarks_dir):
        os.chdir(landmarks_dir)
        fnames=[]
        ids=[]
        masks=[]
        landmarks_list=[]
        points_names=[]
        
        # iteramos en los archivos y rellenamos los 4 primeros
        i=0
        for file in sorted(os.listdir(), key=numericalSort):
            # Check whether file is in text format or not
            if file.endswith(".jpg") or file.endswith(".JPG"):
                image_file=file
                fnames.append(image_file)
                ids.append(i)
                # Construimos las BB
                image = io.imread(image_file)
                if(len(image.shape)<3):
                    #Grey scale to RGB
                    image=np.array([image,image,image])
                    image=np.moveaxis(image, 0, -1)
                i+=1

            if file.endswith(".pts"):
                with open(file) as f:
                    lines=f.readlines()
                    
                count=0
                landmarks=[]
                mask=[]
                for line in lines: 
                    count +=1
                    if(count >= 4 and count<=33):
                        vector=[int(i) for i in line.split()]
                        if(vector[0]==-1):
                            mask.append(0)
                        else:
                            mask.append(1)
                        landmarks.append(vector)
                
                landmarks_list.append(landmarks)
                masks.append(mask)
            
            if file.endswith(".info"):
                with open(file) as f:
                    lines=f.readlines()
                points_names=[i.strip() for i in lines[:len(lines)-1]]
    
        df=pd.DataFrame({
            'fnames':fnames,
            'ra':ids,
            'landmarks_full':landmarks_list,
            'masks': masks
        })
        os.chdir("..")
        return df,points_names


In [3]:
dataframe,names=readLandmarks('./FORENSE_AM')

Vamos a representar en un histograma la distribución de landmarks marcados en las imágenes.

In [4]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

x=list(0 for i in range(30))

for i in dataframe.masks:
    for j in range(30):
        x[j]+=i[j]

plt.figure(figsize=(15,10),dpi=800)
plt.bar(names,x)
plt.xticks(rotation='vertical')
plt.title("Distribución de los landmarks en las imagenes")
plt.savefig('distribucion_landmarks_imagenes.png')
plt.show

findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.
findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.


<function matplotlib.pyplot.show(close=None, block=None)>